In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Documents/programming/BDTools.jl/examples`


In [2]:
Pkg.add(["Plots","Lux","ADTypes","Optimisers","MLUtils","Zygote","HDF5"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/Documents/programming/BDTools.jl/examples/Project.toml`
⌅ [3bd65402] + Optimisers v0.3.4
  No Changes to `~/Documents/programming/BDTools.jl/examples/Manifest.toml`


In [3]:
using HDF5: HDF5
using Plots, Random, Lux, ADTypes, Optimisers, Zygote, Printf, MLUtils, Statistics

In [4]:
struct GroundTruthCleaned
    data::Array{Float64,3}
end
    
function deserialize(filepath::String, ::Type{GroundTruthCleaned})
    HDF5.h5open(filepath, "r") do io
        GroundTruthCleaned(
            io["GroundTruthCleaned/data"] |> read
        )
    end
end

deserialize (generic function with 1 method)

In [5]:
"""
    standardize(data::AbstractArray; dims=1)

Return standardized `data` over dimension `dims`.
"""
function standardize(data::AbstractArray; dims=1)
    μ = mean(data, dims=dims)
    σ = std(data, dims=dims, mean=μ)
    (data.-μ) ./ σ, μ, σ
end

standardize

In [6]:
gt = deserialize("gt_clean.h5", GroundTruthCleaned)
ori64, ori_mean, ori_std = standardize(reshape(gt.data[:,:,1],
    (size(gt.data[:,:,1])[1],1,size(gt.data[:,:,1])[2])))
sim64, sim_mean, sim_std = standardize(reshape(gt.data[:,:,2],
    (size(gt.data[:,:,2])[1],1,size(gt.data[:,:,2])[2])))
ori = Float32.(ori64)
sim = Float32.(sim64)

400×1×1056 Array{Float32, 3}:
[:, :, 1] =
 -0.25204962
  0.8046118
 -1.1035193
 -0.79414064
  1.9682941
  0.5637778
 -0.012230185
 -0.21458794
 -0.07579102
  1.606328
 -0.5129932
 -0.49603918
  1.8115286
  ⋮
  0.60853773
 -0.38985053
 -0.50285214
 -0.4290375
 -0.2008933
  0.90451586
 -0.26445413
  0.938424
 -1.2621317
  1.3216174
  1.6147438
  1.8672619

[:, :, 2] =
  0.46098885
  0.7983072
  0.9000875
 -0.5965039
  1.0699972
 -0.62849957
 -1.6455749
 -0.9128678
 -0.49982587
  1.4695151
 -1.2132338
  1.2019722
 -0.807777
  ⋮
  0.23529959
  0.5053252
 -0.57064825
 -0.05665371
  0.18813378
 -1.2495686
  0.4842252
 -0.3646048
  0.07366619
  2.0850706
  0.35293218
  0.8921558

[:, :, 3] =
  0.2637831
 -0.953591
  1.8452263
  0.82109296
  0.5363555
 -0.87426835
 -0.23479255
 -2.2671652
  0.2375749
  0.2216988
 -0.96207213
 -1.3896456
  0.358906
  ⋮
 -0.8054758
 -0.74574023
  0.6919415
 -0.61785483
 -1.5915744
  1.1232259
  0.41837677
  0.8646118
 -0.68044275
  0.41274798
 -1.5104008
 -0.963

In [7]:
(i_train, o_train), (i_test, o_test) = splitobs((ori, sim); at=0.8)
train_dataloader = DataLoader(collect.((i_train, o_train)); batchsize=8, shuffle=true)
test_dataloader = DataLoader(collect.((i_test, o_test)); batchsize=8, shuffle=true)

27-element DataLoader(::Tuple{Array{Float32, 3}, Array{Float32, 3}}, shuffle=true, batchsize=8)
  with first element:
  (400×1×8 Array{Float32, 3}, 400×1×8 Array{Float32, 3},)

In [8]:
rng = MersenneTwister()
Random.seed!(rng, 12345)

MersenneTwister(12345)

In [9]:
function negr2(y_pred, y_true)
    SS_tot = sum(abs2, vec(y_true) .- mean(vec(y_true)))
    return negr2(y_pred, y_true, SS_tot)
end

function negr2(y_pred, y_true, SS_tot)
    SS_res =  sum(abs2, vec(y_true) .- vec(y_pred))
    r2 = 1 - SS_res/(SS_tot + eps())
    return -r2
end

negr2 (generic function with 2 methods)

In [10]:
loss_negr2 = GenericLossFunction(negr2)
const mseloss_function = MSELoss()

(::GenericLossFunction{typeof(Lux.LossFunctionImpl.l2_distance_loss), typeof(mean)}) (generic function with 2 methods)

In [11]:
train_data = collect(train_dataloader)

106-element Vector{Tuple{Array{Float32, 3}, Array{Float32, 3}}}:
 ([-0.12904055; -0.74002886; … ; -0.15248832; -0.9678636;;; -0.08969547; -0.64527464; … ; -0.23183869; -0.94188476;;; -0.26199153; -0.8739501; … ; -0.14498748; -1.0559232;;; -0.1603327; -0.68400973; … ; -0.3176369; -1.0316557;;; 0.54901; 0.6879873; … ; 0.8415705; 1.31415;;; 0.32267785; 0.85443187; … ; 0.2715317; 1.1193714;;; 0.12830567; 0.65194654; … ; 0.29636148; 0.9916155;;; -0.5439662; -0.7517162; … ; -0.7912847; -1.3204832], [-0.2935141; -1.025746; … ; -0.19840579; -1.2898817;;; 0.503593; -0.51018155; … ; 0.7853488; 0.65165615;;; -0.7109996; -0.6936571; … ; 0.85206914; -0.861804;;; 0.40158135; -0.19193839; … ; 1.1738425; 0.033302955;;; 0.3613174; 0.83169854; … ; -0.76082784; 1.219694;;; -0.22638912; -0.13464166; … ; -1.4221513; -1.751251;;; 0.7480971; -0.06747933; … ; -0.42858213; 0.49713326;;; 0.72072005; -0.15104951; … ; 2.207723; 1.0886638])
 ([-0.73054016; -1.0894914; … ; -0.6148413; -1.4776556;;; -0.16003248; -0.

In [12]:
negr2(train_data[1][1],train_data[2][1])

0.3727925903620586

In [13]:
model = Chain(
        Conv((9,), 1=>18, sigmoid, pad=SamePad()),
        [Chain(Conv((9,), 18=>18, pad=SamePad()), BatchNorm(18, sigmoid)) for _ in 1:6]...,
        Dropout(0.2),
        Conv((1,), 18=>1, pad=SamePad())
    )

Chain(
    layer_1 = Conv((9,), 1 => 18, σ, pad=4),  # 180 parameters
    layer_2 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_3 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_4 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_5 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 37
    ),
    layer_6 = Chain(
        layer_1 = Conv((9,), 18 => 18, pad=4),  # 2_934 parameters
        layer_2 = BatchNorm(18, σ, affine=true, track_stats=true),  # 36 parameters, plus 

In [14]:
ps, st = Lux.setup(rng, model)

((layer_1 = (weight = Float32[-0.105983034; -0.17700903; … ; 0.17689286; 0.5762858;;; -0.030952485; -0.2516598; … ; 0.1556233; -0.19918957;;; 0.013296677; 0.47829136; … ; 0.44485468; -0.038110614;;; … ;;; -0.5703142; -0.3655195; … ; -0.14900751; -0.2651057;;; -0.19353914; 0.2612538; … ; 0.021623876; -0.07029934;;; -0.10583988; 0.5010744; … ; 0.5380823; 0.3065268], bias = Float32[-0.17057855, 0.24381645, 0.10418979, -0.10502354, 0.24592765, -0.30111828, 0.16615231, 0.070468344, 0.08851385, 0.1231846, -0.29632887, -0.21839802, 0.30791894, 0.1067009, -0.2811664, 0.09673294, 0.26899657, -0.15632947]), layer_2 = (layer_1 = (weight = Float32[0.026080474 -0.055631038 … -0.123914324 -0.031135645; 0.13578852 0.05806497 … 0.09317774 -0.09072603; … ; 0.04616294 0.07360015 … 0.0073334333 -0.07687018; 0.023676682 0.12932956 … -0.052710045 0.07514027;;; 0.009319014 -0.018593287 … 0.13592397 0.04805164; -0.014001005 0.025715278 … -0.04884728 0.10543372; … ; 0.056692075 0.065480456 … 0.119357854 0.075

In [15]:
opt = Adam(1e-5)

Adam(1.0e-5, (0.9, 0.999), 1.0e-8)

In [16]:
tstate = Training.TrainState(model, ps, st, opt)

TrainState
    model: Chain{@NamedTuple{layer_1::Conv{typeof(σ), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::BatchNorm{typeof(σ), Float32, Float32, Int64, typeof(zeros32), typeof(ones32), Static.True, Static.True}}, Nothing}, layer_3::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::BatchNorm{typeof(σ), Float32, Float32, Int64, typeof(zeros32), typeof(ones32), Static.True, Static.True}}, Nothing}, layer_4::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, S

In [17]:
function main(tstate::Training.TrainState, epochs)
    loss = 1e18
    localloss = 100.0
    best_tstate = nothing
    for epoch in 1:epochs
        for (x,y) in train_dataloader
#            println(size(x))
            _, l, _, tstate = Training.single_train_step!(AutoZygote(), mseloss_function, (x,y), tstate)
            localloss = l
            if l<loss
                loss = l
                best_tstate = tstate
            end
        end
        loss_test = mseloss_function(Lux.apply(best_tstate.model, i_test, best_tstate.parameters, best_tstate.states)[1], o_test)
        fidelity = cor(vec(Lux.apply(tstate.model, i_test, tstate.parameters, tstate.states)[1]),vec(o_test))
        @printf "Epoch: %3d \t Current Loss: %.5g \t Best Loss: %.5g \t Test Loss: %.5g \t Fidelity: %.5g\n" epoch localloss loss loss_test fidelity
    end
    return best_tstate
end

tstate = main(tstate, 100)

┌ Warning: `training` is set to `Val{true}()` but is not being used within an autodiff call (gradient, jacobian, etc...). This will be slow. If you are using a `Lux.jl` model, set it to inference (test) mode using `LuxCore.testmode`. Reliance on this behavior is discouraged, and is not guaranteed by Semantic Versioning, and might be removed without a deprecation cycle. It is recommended to fix this issue in your code.
└ @ LuxLib.Utils ~/.julia/packages/LuxLib/ru5RQ/src/utils.jl:314


Epoch:   1 	 Current Loss: 1.8676 	 Best Loss: 1.8661 	 Test Loss: 1.8755 	 Fidelity: -0.0056623
Epoch:   2 	 Current Loss: 1.8468 	 Best Loss: 1.7513 	 Test Loss: 1.8053 	 Fidelity: 0.073875
Epoch:   3 	 Current Loss: 1.7458 	 Best Loss: 1.6456 	 Test Loss: 1.7658 	 Fidelity: 0.10861
Epoch:   4 	 Current Loss: 1.7824 	 Best Loss: 1.6413 	 Test Loss: 1.7398 	 Fidelity: 0.12677
Epoch:   5 	 Current Loss: 1.6719 	 Best Loss: 1.5851 	 Test Loss: 1.7178 	 Fidelity: 0.13373
Epoch:   6 	 Current Loss: 1.7342 	 Best Loss: 1.5247 	 Test Loss: 1.6959 	 Fidelity: 0.1325
Epoch:   7 	 Current Loss: 1.7416 	 Best Loss: 1.5247 	 Test Loss: 1.6761 	 Fidelity: 0.13866
Epoch:   8 	 Current Loss: 1.6162 	 Best Loss: 1.5247 	 Test Loss: 1.6568 	 Fidelity: 0.14417
Epoch:   9 	 Current Loss: 1.5404 	 Best Loss: 1.5061 	 Test Loss: 1.6411 	 Fidelity: 0.13924
Epoch:  10 	 Current Loss: 1.5727 	 Best Loss: 1.5019 	 Test Loss: 1.6203 	 Fidelity: 0.14016
Epoch:  11 	 Current Loss: 1.6467 	 Best Loss: 1.4679 	 T

TrainState
    model: Chain{@NamedTuple{layer_1::Conv{typeof(σ), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::BatchNorm{typeof(σ), Float32, Float32, Int64, typeof(zeros32), typeof(ones32), Static.True, Static.True}}, Nothing}, layer_3::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, Static.False}, layer_2::BatchNorm{typeof(σ), Float32, Float32, Int64, typeof(zeros32), typeof(ones32), Static.True, Static.True}}, Nothing}, layer_4::Chain{@NamedTuple{layer_1::Conv{typeof(identity), Int64, Int64, Tuple{Int64}, Tuple{Int64}, Tuple{Int64, Int64}, Tuple{Int64}, Int64, Nothing, Nothing, Static.True, S

In [85]:
cor(vec(i_test),vec(o_test))

0.22671475f0